#Setup

In [31]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.pyplot import figure
import seaborn as sns
import pandas as pd
import operator
from math import log
from pandas import Series, DataFrame
from pylab import rcParams
import pyreadr

import sklearn
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold # import KFold
from sklearn import datasets

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support

from matplotlib.lines import Line2D
import re
import pickle

In [32]:
passing_grades = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D",
                  "D-", "S", "CR"]
letter_grades = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D",
                  "D-", "NP", "W"]
passing_pattern = r"A[+-]*|B[+-]*|C[+-]*|D[+-]*|S|CR"
success_grades = ["A+", "A", "A-", "B+"]
DATA_PATH = "../data"

# Dataset of the last 5 years record

In [40]:
raw_data = pd.read_csv('../data/clean_full.csv', header=0)
raw_data = raw_data.drop(['Unnamed: 0'], axis=1)
raw_data.columns

Index(['acad_career', 'strm', 'stdnt_enrl_status', 'unt_taken', 'unt_billing',
       'crse_grade_input', 'earn_credit', 'emplid', 'subject', 'catalog_nbr',
       'crse_acad_org', 'sex'],
      dtype='object')

Winter is 4, Spring is 6, Fall is 2, Summer is 8?
e.g. 1198 is 18-19, summer quarter. 1196 is 18-19, spring quarter

In [37]:
def get_course_name (row):
    if row['subject'] != None and row['catalog_nbr'] != None:
        return row['subject']+row['catalog_nbr']
    return ""
def dropped_function (row):
    if row['stdnt_enrl_status'] == 'D':
        return 1
    return 0

In [58]:
def getStudentFeatures(subject, number, select_majors=False):
    course_entries = raw_data.loc[(raw_data['subject'] == subject) & (raw_data['catalog_nbr'] == number) & # all course entries that are about the certain course we want
                                  (raw_data['strm'] >= 1162) & # taken in 15-16 fall or later
                                  ((raw_data['crse_grade_input'].isin(letter_grades)) | (raw_data['strm'] == 1204)) & #Grade needs to be a letter or it's the latest quarter (nan)
                                  (raw_data['strm'] != 1168) & (raw_data['strm'] != 1178) & (raw_data['strm'] != 1188) &
                                  (raw_data['strm'] != 1198) & (raw_data['stdnt_enrl_status'] != 'D')] # not dropped and not summer
#     return course_entries
    students = course_entries.emplid # Students from the course entries
    students_prev_classes = raw_data.loc[raw_data['emplid'].isin(students)] # Other classes taken by these students
    if select_majors:
        students_prev_classes = students_prev_classes.loc[students_prev_classes['subject'].isin(['CME', 'CS', 'EE', 'MATH', 'STATS'])]
    possible_prev_classes = students_prev_classes.apply(lambda row: get_course_name(row), axis=1) # List of courses taken by these students
    possible_prev_classes = possible_prev_classes.unique().tolist() # Focus on unique courses
    possible_prev_classes.append("strm") # For now add term to the data of when the course was taken
    possible_prev_classes.append("female") # Add gender
    possible_prev_classes.append("emplid") # Add emplid
    possible_prev_classes.append("y") # Add the target to the data. Will split it later
    a = np.zeros(shape=(len(course_entries),len(possible_prev_classes))) # all 0 initialized features matrix. +2 for female and strm
    dataset = pd.DataFrame(a,columns=possible_prev_classes) # same as above, but in pandas to allow indexing by course name
    print(len(course_entries))
    print('looping')
    for counter, idx in enumerate(course_entries.index): # loop over all class entried
        print("student: ", counter, " of ", len(course_entries))
        grade = course_entries.loc[[idx]]['crse_grade_input']
        gradeStr = grade.tolist()[0]
        if (gradeStr == "A+" or gradeStr == "A" or
          gradeStr == "A-" or gradeStr == "B+"):
            dataset['y'][counter] = 1
        # fill predictor matrix X
        std_id = course_entries.loc[[idx]]['emplid'] # Get the student id
        quarter = course_entries.loc[[idx]]['strm'] # Get the term
        female = course_entries.loc[[idx]]['sex'] # Get the student gender
        student_courses = students_prev_classes.loc[students_prev_classes['emplid'] == std_id.tolist()[0]] # Get all courses the student took
        student_courses = student_courses.loc[student_courses['strm'] < quarter.tolist()[0]] # Filter the courses to be only before the course of interest
        if select_majors:
            student_courses = student_courses.loc[student_courses['subject'].isin(['CME', 'CS', 'EE', 'MATH', 'STATS'])]
        if (female.tolist()[0] == 'M'):
            dataset["female"][counter] = 0
        else:
            dataset["female"][counter] = 1
        dataset["strm"][counter] = quarter.tolist()[0]
        dataset["emplid"][counter] = std_id.tolist()[0]
        for j in student_courses.index:# Loop over the student's courses
            course_name = student_courses.loc[[j]]['subject']+student_courses.loc[[j]]['catalog_nbr'] # Get the course name
            grade = student_courses.loc[[j]]['crse_grade_input']
            gradeStr = grade.tolist()[0]
            if (gradeStr == "A+" or gradeStr == "A" or
                gradeStr == "A-" or gradeStr == "B+"):
                dataset.iloc[counter, dataset.columns.get_loc(course_name.tolist()[0])] = 1 # Set the course name in X to the new value (or if it was already passed keep the value)
#             dev_break = True
    return dataset

In [59]:
#dataset includes a y column, which is the target. It also includes strm and emplid columns that should
#be dropped before running any prediction
dataset_110 = getStudentFeatures('CS', '110')

1683
looping
student:  0  of  1683
student:  1  of  1683
student:  2  of  1683
student:  3  of  1683


/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42:

student:  4  of  1683
student:  5  of  1683
student:  6  of  1683


/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


student:  7  of  1683
student:  8  of  1683
student:  9  of  1683
student:  10  of  1683
student:  11  of  1683
student:  12  of  1683
student:  13  of  1683
student:  14  of  1683
student:  15  of  1683
student:  16  of  1683
student:  17  of  1683
student:  18  of  1683
student:  19  of  1683
student:  20  of  1683
student:  21  of  1683
student:  22  of  1683
student:  23  of  1683
student:  24  of  1683
student:  25  of  1683
student:  26  of  1683
student:  27  of  1683
student:  28  of  1683
student:  29  of  1683
student:  30  of  1683
student:  31  of  1683
student:  32  of  1683
student:  33  of  1683
student:  34  of  1683
student:  35  of  1683
student:  36  of  1683
student:  37  of  1683
student:  38  of  1683
student:  39  of  1683
student:  40  of  1683
student:  41  of  1683
student:  42  of  1683
student:  43  of  1683
student:  44  of  1683
student:  45  of  1683
student:  46  of  1683
student:  47  of  1683
student:  48  of  1683
student:  49  of  1683
student:  50  

student:  353  of  1683
student:  354  of  1683
student:  355  of  1683
student:  356  of  1683
student:  357  of  1683
student:  358  of  1683
student:  359  of  1683
student:  360  of  1683
student:  361  of  1683
student:  362  of  1683
student:  363  of  1683
student:  364  of  1683
student:  365  of  1683
student:  366  of  1683
student:  367  of  1683
student:  368  of  1683
student:  369  of  1683
student:  370  of  1683
student:  371  of  1683
student:  372  of  1683
student:  373  of  1683
student:  374  of  1683
student:  375  of  1683
student:  376  of  1683
student:  377  of  1683
student:  378  of  1683
student:  379  of  1683
student:  380  of  1683
student:  381  of  1683
student:  382  of  1683
student:  383  of  1683
student:  384  of  1683
student:  385  of  1683
student:  386  of  1683
student:  387  of  1683
student:  388  of  1683
student:  389  of  1683
student:  390  of  1683
student:  391  of  1683
student:  392  of  1683
student:  393  of  1683
student:  394  o

student:  695  of  1683
student:  696  of  1683
student:  697  of  1683
student:  698  of  1683
student:  699  of  1683
student:  700  of  1683
student:  701  of  1683
student:  702  of  1683
student:  703  of  1683
student:  704  of  1683
student:  705  of  1683
student:  706  of  1683
student:  707  of  1683
student:  708  of  1683
student:  709  of  1683
student:  710  of  1683
student:  711  of  1683
student:  712  of  1683
student:  713  of  1683
student:  714  of  1683
student:  715  of  1683
student:  716  of  1683
student:  717  of  1683
student:  718  of  1683
student:  719  of  1683
student:  720  of  1683
student:  721  of  1683
student:  722  of  1683
student:  723  of  1683
student:  724  of  1683
student:  725  of  1683
student:  726  of  1683
student:  727  of  1683
student:  728  of  1683
student:  729  of  1683
student:  730  of  1683
student:  731  of  1683
student:  732  of  1683
student:  733  of  1683
student:  734  of  1683
student:  735  of  1683
student:  736  o

student:  1035  of  1683
student:  1036  of  1683
student:  1037  of  1683
student:  1038  of  1683
student:  1039  of  1683
student:  1040  of  1683
student:  1041  of  1683
student:  1042  of  1683
student:  1043  of  1683
student:  1044  of  1683
student:  1045  of  1683
student:  1046  of  1683
student:  1047  of  1683
student:  1048  of  1683
student:  1049  of  1683
student:  1050  of  1683
student:  1051  of  1683
student:  1052  of  1683
student:  1053  of  1683
student:  1054  of  1683
student:  1055  of  1683
student:  1056  of  1683
student:  1057  of  1683
student:  1058  of  1683
student:  1059  of  1683
student:  1060  of  1683
student:  1061  of  1683
student:  1062  of  1683
student:  1063  of  1683
student:  1064  of  1683
student:  1065  of  1683
student:  1066  of  1683
student:  1067  of  1683
student:  1068  of  1683
student:  1069  of  1683
student:  1070  of  1683
student:  1071  of  1683
student:  1072  of  1683
student:  1073  of  1683
student:  1074  of  1683


student:  1363  of  1683
student:  1364  of  1683
student:  1365  of  1683
student:  1366  of  1683
student:  1367  of  1683
student:  1368  of  1683
student:  1369  of  1683
student:  1370  of  1683
student:  1371  of  1683
student:  1372  of  1683
student:  1373  of  1683
student:  1374  of  1683
student:  1375  of  1683
student:  1376  of  1683
student:  1377  of  1683
student:  1378  of  1683
student:  1379  of  1683
student:  1380  of  1683
student:  1381  of  1683
student:  1382  of  1683
student:  1383  of  1683
student:  1384  of  1683
student:  1385  of  1683
student:  1386  of  1683
student:  1387  of  1683
student:  1388  of  1683
student:  1389  of  1683
student:  1390  of  1683
student:  1391  of  1683
student:  1392  of  1683
student:  1393  of  1683
student:  1394  of  1683
student:  1395  of  1683
student:  1396  of  1683
student:  1397  of  1683
student:  1398  of  1683
student:  1399  of  1683
student:  1400  of  1683
student:  1401  of  1683
student:  1402  of  1683


In [60]:
import collections
collections.Counter(dataset_110.y)

Counter({0.0: 1014, 1.0: 669})

# Try sklearn logistic regression

In [61]:
from sklearn.linear_model import LogisticRegression

In [66]:
# Use the X predictor matrix and y outcome vector from 
# getStudentFeatures, fit a logistic regression model and return
# the list of coefficients, default sorted by absolute value

# Params: sort = ["pos", "neg", "abs"] to sort by largest positive, negative, or
# absolute value of coefficients
def get_coefs(X_train, X_test, y_train, y_test, sort = "abs"):
    model = LogisticRegression(solver="lbfgs")
    model.fit(X_train, y_train)
    train_score = model.score(X_train,y_train) 
    test_score = model.score(X_test,y_test)
    print("training accuracy: %s" % train_score)
    print("test accuracy: %s" % test_score)
    coefs = [(X_test.columns[i], model.coef_[0][i]) for i in range(len(X_test.columns))]
    if sort:
        if sort == "pos":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
        if sort == "neg":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = False)
        if sort == "abs":
            coefs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
    return coefs

In [71]:
def get_and_split_data(subject, course_nbr, test_quarters):
    #For now, dropping this quarter
    dataset = getStudentFeatures('CS', '110')
    dataset_grades = dataset.loc[dataset_['strm'] < 1204.0]
    dataset_train = dataset_grades.loc[~dataset_grades['strm'].isin(test_quarters)]
    dataset_test = dataset_grades.loc[dataset_grades['strm'].isin(test_quarters)]

    dataset_train = dataset_train.drop(["strm", "emplid"], axis=1)
    dataset_test = dataset_test.drop(["strm", "emplid"], axis=1)

    X_train = dataset_train.drop(["y"], axis=1)
    y_train = dataset_train["y"]
    X_test = dataset_test.drop(["y"], axis=1)
    y_test = dataset_test["y"]
    return X_train, X_test, y_train, y_test

In [72]:
X_train, X_test, y_train, y_test = get_and_split_data('CS', '110', [1194.0, 1196.0])
CS110coefs = get_coefs(X_train, X_test, y_train, y_test, 'abs')
CS110coefs[:100]

1683
looping
student:  0  of  1683
student:  1  of  1683
student:  2  of  1683
student: 

/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42:

 3  of  1683
student:  4  of  1683
student:  5  of  1683


/Users/Abood/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


student:  6  of  1683
student:  7  of  1683
student:  8  of  1683
student:  9  of  1683
student:  10  of  1683
student:  11  of  1683
student:  12  of  1683
student:  13  of  1683
student:  14  of  1683
student:  15  of  1683
student:  16  of  1683
student:  17  of  1683
student:  18  of  1683
student:  19  of  1683
student:  20  of  1683
student:  21  of  1683
student:  22  of  1683
student:  23  of  1683
student:  24  of  1683
student:  25  of  1683
student:  26  of  1683
student:  27  of  1683
student:  28  of  1683
student:  29  of  1683
student:  30  of  1683
student:  31  of  1683
student:  32  of  1683
student:  33  of  1683
student:  34  of  1683
student:  35  of  1683
student:  36  of  1683
student:  37  of  1683
student:  38  of  1683
student:  39  of  1683
student:  40  of  1683
student:  41  of  1683
student:  42  of  1683
student:  43  of  1683
student:  44  of  1683
student:  45  of  1683
student:  46  of  1683
student:  47  of  1683
student:  48  of  1683
student:  49  o

student:  353  of  1683
student:  354  of  1683
student:  355  of  1683
student:  356  of  1683
student:  357  of  1683
student:  358  of  1683
student:  359  of  1683
student:  360  of  1683
student:  361  of  1683
student:  362  of  1683
student:  363  of  1683
student:  364  of  1683
student:  365  of  1683
student:  366  of  1683
student:  367  of  1683
student:  368  of  1683
student:  369  of  1683
student:  370  of  1683
student:  371  of  1683
student:  372  of  1683
student:  373  of  1683
student:  374  of  1683
student:  375  of  1683
student:  376  of  1683
student:  377  of  1683
student:  378  of  1683
student:  379  of  1683
student:  380  of  1683
student:  381  of  1683
student:  382  of  1683
student:  383  of  1683
student:  384  of  1683
student:  385  of  1683
student:  386  of  1683
student:  387  of  1683
student:  388  of  1683
student:  389  of  1683
student:  390  of  1683
student:  391  of  1683
student:  392  of  1683
student:  393  of  1683
student:  394  o

student:  694  of  1683
student:  695  of  1683
student:  696  of  1683
student:  697  of  1683
student:  698  of  1683
student:  699  of  1683
student:  700  of  1683
student:  701  of  1683
student:  702  of  1683
student:  703  of  1683
student:  704  of  1683
student:  705  of  1683
student:  706  of  1683
student:  707  of  1683
student:  708  of  1683
student:  709  of  1683
student:  710  of  1683
student:  711  of  1683
student:  712  of  1683
student:  713  of  1683
student:  714  of  1683
student:  715  of  1683
student:  716  of  1683
student:  717  of  1683
student:  718  of  1683
student:  719  of  1683
student:  720  of  1683
student:  721  of  1683
student:  722  of  1683
student:  723  of  1683
student:  724  of  1683
student:  725  of  1683
student:  726  of  1683
student:  727  of  1683
student:  728  of  1683
student:  729  of  1683
student:  730  of  1683
student:  731  of  1683
student:  732  of  1683
student:  733  of  1683
student:  734  of  1683
student:  735  o

student:  1035  of  1683
student:  1036  of  1683
student:  1037  of  1683
student:  1038  of  1683
student:  1039  of  1683
student:  1040  of  1683
student:  1041  of  1683
student:  1042  of  1683
student:  1043  of  1683
student:  1044  of  1683
student:  1045  of  1683
student:  1046  of  1683
student:  1047  of  1683
student:  1048  of  1683
student:  1049  of  1683
student:  1050  of  1683
student:  1051  of  1683
student:  1052  of  1683
student:  1053  of  1683
student:  1054  of  1683
student:  1055  of  1683
student:  1056  of  1683
student:  1057  of  1683
student:  1058  of  1683
student:  1059  of  1683
student:  1060  of  1683
student:  1061  of  1683
student:  1062  of  1683
student:  1063  of  1683
student:  1064  of  1683
student:  1065  of  1683
student:  1066  of  1683
student:  1067  of  1683
student:  1068  of  1683
student:  1069  of  1683
student:  1070  of  1683
student:  1071  of  1683
student:  1072  of  1683
student:  1073  of  1683
student:  1074  of  1683


student:  1364  of  1683
student:  1365  of  1683
student:  1366  of  1683
student:  1367  of  1683
student:  1368  of  1683
student:  1369  of  1683
student:  1370  of  1683
student:  1371  of  1683
student:  1372  of  1683
student:  1373  of  1683
student:  1374  of  1683
student:  1375  of  1683
student:  1376  of  1683
student:  1377  of  1683
student:  1378  of  1683
student:  1379  of  1683
student:  1380  of  1683
student:  1381  of  1683
student:  1382  of  1683
student:  1383  of  1683
student:  1384  of  1683
student:  1385  of  1683
student:  1386  of  1683
student:  1387  of  1683
student:  1388  of  1683
student:  1389  of  1683
student:  1390  of  1683
student:  1391  of  1683
student:  1392  of  1683
student:  1393  of  1683
student:  1394  of  1683
student:  1395  of  1683
student:  1396  of  1683
student:  1397  of  1683
student:  1398  of  1683
student:  1399  of  1683
student:  1400  of  1683
student:  1401  of  1683
student:  1402  of  1683
student:  1403  of  1683


NameError: name 'dataset_' is not defined


# **Lasso Model**

In [ ]:
#This is a bad model. Leave here for reference
def get_coefs_lasso(X_train, X_test, y_train, y_test):
  model = Lasso(alpha=0.0001,precompute=True,max_iter=1000,positive=True, random_state=9999, selection='random')
  model.fit(X_train, y_train)
  coefs = [(X_train.columns[i], model.coef_[i]) for i in range(len(X_train.columns))]
  coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
  train_score = model.score(X_train,y_train)
  test_score = model.score(X_test,y_test)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)

  return coefs

In [ ]:
coefs = get_coefs_lasso(X_train, X_test, y_train, y_test)
coefs[:20]

# **SVM**

In [ ]:
from sklearn.svm import SVC
def get_coefs_svm(X_train, X_test, y_train, y_test):
  model = SVC(kernel='rbf',gamma='scale')
  model.fit(X_train, y_train)
  train_score = model.score(X_train,y_train)
  test_score = model.score(X_test,y_test)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)


In [ ]:
X = pd.read_pickle(DATA_PATH+"/CS142_X.pkl")
y = np.load(DATA_PATH+"/CS142_y.npy")
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
coefs = get_coefs_svm(X_train, X_test, y_train, y_test)

# Trying Recursive Feature Elimination

In [55]:
from sklearn.feature_selection import RFE
def run_RFE(X, y, sort = "abs"):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LogisticRegression(solver="lbfgs")
    selector = RFE(model, 10, step=1)
    selector.fit(X_train, y_train)
    train_score = selector.score(X_test,y_test) 
    print("training accuracy: %s" % train_score)
#     coefs = [(X.columns[i], model.coef_[0][i]) for i in range(len(X.columns))]
#     if sort:
#         if sort == "pos":
#             coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
#         if sort == "neg":
#             coefs = sorted(coefs, key = lambda x: x[1], reverse = False)
#         if sort == "abs":
#             coefs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
#     return coefs

In [56]:
run_RFE(X_mod, Y_mod, 'abs')

KeyboardInterrupt: 

Things to try:
- PCA
- Collaborative filtering to detect feature vectors of classes, then cluster them? (Is this just more complicated PCA?)
- Adjust $\lambda$ for normalization
- Change it so the coefficients are all $e^{\text{coef}}$ (all positive), requires some kind of transformation to log space?
- Full dataset instead of just CS, find predictors in other departments
- Weighted regression or some kind of significance indicator to account for the predictors with very few data points

Interpretations:
- What do we do with negative coefficients?

# **Compare different Models**

# Older Stuff: Might need changing to work with our new data

Haven't changed from the old model

In [ ]:
#Haven't changed from the old model
def runDifferentModels(X,y):
  # split X and y into training and testing sets
  X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, y, test_size=0.3)
  X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)
  print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)
  # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  #"Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process","Decision Tree", "Random Forest", "Neural Net", "AdaBoost","Naive Bayes", "QDA"
  models = []
  models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
  models.append(('LDA', LinearDiscriminantAnalysis()))
  models.append(('GP', GaussianProcessClassifier()))
  models.append(('RF', RandomForestClassifier()))
  models.append(('NN', MLPClassifier()))
  models.append(('AB', AdaBoostClassifier()))
  models.append(('QDA', QuadraticDiscriminantAnalysis()))
  models.append(('CART', DecisionTreeClassifier()))
  models.append(('NB', GaussianNB()))
  models.append(('K_M', KNeighborsClassifier(metric = 'minkowski')))
  models.append(('K_C', KNeighborsClassifier(metric = 'chebyshev')))
  models.append(('K_E', KNeighborsClassifier(metric = 'euclidean')))
  models.append(('K_Man', KNeighborsClassifier(metric = 'manhattan')))
  models.append(('K_Mat', KNeighborsClassifier(metric = 'matching')))
  models.append(('K_J', KNeighborsClassifier(metric = 'jaccard')))
  models.append(('K_D', KNeighborsClassifier(metric = 'dice')))
  models.append(('K_K', KNeighborsClassifier(metric = 'kulsinski')))
  models.append(('S_Lin', SVC(kernel='linear')))
  models.append(('S_RBF', SVC(kernel='rbf')))
  models.append(('S_Sig', SVC(kernel="sigmoid")))
  models.append(('S_Pol', SVC(kernel="poly")))
  

  # Evaluate each model through 10-fold cross-validation
  results = []
  names = []
  number = 1
  for name, model in models:
      kfold = KFold(n_splits=10, random_state=42)
      cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
      results.append(cv_results)
      names.append(name)
      msg = "%i) %s: Mean = %f with std = (%f)" % (number, name, cv_results.mean(), cv_results.std())
      number += 1
      print(msg)
  # Compare Algorithms
  matplotlib.rcParams.update({'font.size': 40})
  fig = plt.figure()
  fig = plt.figure(figsize=(28,20))
  plt.title('Algorithm Comparison')
  ax = fig
  medianprops = {'color': 'magenta', 'linewidth': 6}
  boxprops = {'color': 'black', 'linewidth': 4, 'linestyle': '-'}
  whiskerprops = {'color': 'black', 'linewidth': 4, 'linestyle': '-'}
  capprops = {'color': 'black', 'linewidth': 4, 'linestyle': '-'}
  flierprops = {'color': 'black', 'marker': 'x'}
  plt.boxplot(results,
           medianprops=medianprops,
           boxprops=boxprops,
           whiskerprops=whiskerprops,
           capprops=capprops,
           flierprops=flierprops)
#   ax.set_xticklabels(names)
  plt.xlabel('Model Number', fontsize=40)
  plt.ylabel('Accuracy', fontsize=40)
  plt.savefig(DATA_PATH+'/visualizations/comparison_110_B+.png')

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X_110)

# X_train (10 input features, 70% of full dataset), X_val (10 input features, 15% of full dataset), X_test (10 input features, 15% of full dataset)
# Y_train (1 label, 70% of full dataset), Y_val (1 label, 15% of full dataset), Y_test (1 label, 15% of full dataset)
X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X_scale, y_110, test_size=0.3)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

In [ ]:
runDifferentModels(X_110,y_110)

# **PCA**

Haven't changed from the old model

In [ ]:

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 

pca = PCA(n_components=50)
principalComponents = pca.fit_transform(X_110)
principalDf = pd.DataFrame(data = principalComponents)

CS110coefs = get_coefs(principalDf, y_110)
CS110coefs[:20]
pca = PCA().fit(X_110)
#Plotting the Cumulative Summation of the Explained Variance
matplotlib.rcParams.update({'font.size': 40})
fig = plt.figure()
fig = plt.figure(figsize=(28,20))
plt.plot(np.cumsum(pca.explained_variance_ratio_), lw=7)
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)') #for each component
plt.title('Dataset Explained Variance')
plt.savefig(DATA_PATH+"/visualizations/PCA_110_B+.png")

# **Baseline Analysis**

Haven't changed from the old model

In [ ]:
import pickle
DATA_PATH = "../data"

In [ ]:
#plot learning curves
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    matplotlib.rcParams.update({'font.size': 40})
    fig = plt.figure()
    fig = plt.figure(figsize=(28,20))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score", lw=7)
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score", lw=7)

    plt.legend(loc="best")
    return plt

In [ ]:
def plot_validation_curves(title, param_name, param_range, train_scores, test_scores):
  train_scores_mean = np.mean(train_scores, axis=1)
  train_scores_std = np.std(train_scores, axis=1)
  test_scores_mean = np.mean(test_scores, axis=1)
  test_scores_std = np.std(test_scores, axis=1)
  
  matplotlib.rcParams.update({'font.size': 40})
  fig = plt.figure()
  fig = plt.figure(figsize=(28,20))
  plt.title(title)
  plt.xlabel(param_name)
  plt.ylabel("Score")
  lw = 7
  
  plt.plot(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
  plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
  plt.plot(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
  plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
  plt.legend(loc="best")
  return plt

In [ ]:
#plot validation curves
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve

# X = pd.read_pickle(DATA_PATH+"/CS106B_X_plus.pkl")
# y = np.load(DATA_PATH+"/CS106B_y_plus.npy")

param_range= [50, 100, 200, 400]
estimator = LogisticRegression(solver="lbfgs", multi_class="multinomial")
train_scores, test_scores = validation_curve(estimator, X_230, y_230, param_name="max_iter", param_range=param_range, cv=5, scoring="accuracy", n_jobs=8)

title="Validation Curve of CS230 with Logistic Regression"
plot_validation_curves(title,"max_iter",param_range,train_scores,test_scores)
plt.savefig(DATA_PATH+"/visualizations/validation_curve_230_B+.png")

# GLENN: Reducing number of coefficients (Lasso, PCA, and other attempts)

In [ ]:
# need a different library to get p-values
import statsmodels.api as sm

cs_ugrad_plus = pd.read_csv('/content/drive/My Drive/CS129 Carta project/data/cs_ug_plus.csv', header=0)

In [ ]:
X, y = getStudentFeatures2("CS110", cs_ugrad_plus)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# logit_model = sm.Logit(y, X)
# result = logit_model.fit()
# print(result.summary())

Doesn't work (singular matrix). Let's try LASSO logistic regression.

In [ ]:
from pprint import pprint

model = LogisticRegression(penalty="l1", solver="saga", C=1, max_iter=300) 
model.fit(X_train, y_train)
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)
coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
coefs_nonzero = sum([x[1] != 0 for x in coefs])
coefs_zero = sum(x[1] == 0 for x in coefs)
print("training accuracy: %s" % train_score)
print("test accuracy: %s" % test_score)
print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))

print("Largest absolute coefficients:")
pprint(coefs_abs[:10])
print("Largest positive coefficients:")
pprint(coefs_pos[:10])
print("Largest negative coefficients:")
pprint(coefs_neg[:10])


In [ ]:
model = LogisticRegression(penalty="l1", solver="saga", C=0.1, max_iter=300) 
model.fit(X_train, y_train)
train_score = model.score(X_train,y_train)
test_score = model.score(X_test,y_test)
coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
coefs_nonzero = sum([x[1] != 0 for x in coefs])
coefs_zero = sum(x[1] == 0 for x in coefs)
print("training accuracy: %s" % train_score)
print("test accuracy: %s" % test_score)
print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))

print("Largest absolute coefficients:")
pprint(coefs_abs[:10])
print("Largest positive coefficients:")
pprint(coefs_pos[:10])
print("Largest negative coefficients:")
pprint(coefs_neg[:10])

In [ ]:
for i in [10, 3, 1, 0.3, 0.1, 0.03, 0.01]:
  model = LogisticRegression(penalty="l1", solver="saga", C=i, max_iter=300) 
  model.fit(X_train, y_train)
  train_score = model.score(X_train, y_train)
  test_score = model.score(X_test, y_test)
  coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
  coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
  coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
  coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
  coefs_nonzero = sum([x[1] != 0 for x in coefs])
  coefs_zero = sum(x[1] == 0 for x in coefs)
  print("***** C = %f *****" % i)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)
  print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))
  print("Largest absolute coefficients:")
  pprint(coefs_abs[:3])
  print("Largest positive coefficients:")
  pprint(coefs_pos[:3])
  print("Largest negative coefficients:")
  pprint(coefs_neg[:3])


In [ ]:
X, y = getStudentFeatures2("CS107", cs_ugrad_plus)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

for i in [10, 3, 1, 0.3, 0.1, 0.03, 0.01]:
  model = LogisticRegression(penalty="l1", solver="saga", C=i, max_iter=300) 
  model.fit(X_train, y_train)
  train_score = model.score(X_train, y_train)
  test_score = model.score(X_test, y_test)
  coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
  coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
  coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
  coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
  coefs_nonzero = sum([x[1] != 0 for x in coefs])
  coefs_zero = sum(x[1] == 0 for x in coefs)
  print("***** C = %f *****" % i)
  print("training accuracy: %s" % train_score)
  print("test accuracy: %s" % test_score)
  print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))
  print("Largest absolute coefficients:")
  pprint(coefs_abs[:3])
  print("Largest positive coefficients:")
  pprint(coefs_pos[:3])
  print("Largest negative coefficients:")
  pprint(coefs_neg[:3])

In [ ]:
# accuracy seems weird, check ground truth

print(sum([score == 1 for score in y_train]) / len(y_train))

This model isn't doing much more than just predicting based on the intercept.

# Whole dataset

In [ ]:
cs_ugrad = pd.read_csv('/content/drive/My Drive/CS129 Carta project/data/cs_ug_plus.csv', header=0)
top50courses = cs_ugrad.groupby('course_name').count().nlargest(50, 'strm').index.values
topCourses = np.delete(top50courses,[8,14,18,29,30,38,39,40,45])
topCourses = np.append(topCourses,'CS230')

In [ ]:
#for topCourses
results = []

for course in topCourses:
  X,y = getStudentFeatures(course)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = LogisticRegression(solver="lbfgs")
  model.fit(X_train, y_train)
  #study coefficients
  coefs = [(X.columns[i], model.coef_[0][i]) for i in range(len(X.columns))]
  coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
  coefs_top20 = coefs[:20]
  prereqInCoef = PrereqPredicted(course, coefs_top20)

  #model evaluation
  train_score = model.score(X_train, y_train)
  y_pred_train = model.predict(X_train)
  train_precision, train_recall, train_fscore, train_support = precision_recall_fscore_support(y_train, y_pred_train, average="binary")
  test_score = model.score(X_test, y_test)
  y_pred_test = model.predict(X_test)
  test_precision, test_recall, test_fscore, test_support = precision_recall_fscore_support(y_test, y_pred_test, average="binary")
  
  #save results
  result = {
    'course_name': course, 
    'train_accuracy': train_score, 
    'train_precision': train_precision, 
    'train_recall': train_recall, 
    'train_fscore': train_fscore, 
    'test_accuracy': test_score, 
    'test_precision': test_precision, 
    'test_recall': test_recall, 
    'test_fscore': test_fscore,
    'official_prereq_in_coefs': prereqInCoef,
    'coefs': coefs_top20
  }
  print(result)
  results.append(result)


results = pd.DataFrame(results)
results.to_pickle(DATA_PATH+"/topResults.pkl")

In [ ]:
results.to_csv(DATA_PATH+"/topResults.csv")

In [ ]:
results.to_pickle(DATA_PATH+"/topResultsWith230.pkl")

In [ ]:
cs_ugrad = pd.read_csv('/content/drive/My Drive/CS129 Carta project/data/cs_ug_plus.csv', header=0)
top50courses = cs_ugrad.groupby('course_name').count().nlargest(50, 'strm').index.values
topCourses = np.delete(top50courses,[8,14,18,29,30,38,39,40,45])
topCourses = np.append(topCourses,'CS230')

#encode the data for scatter plot
def encode_course(course):
    encode = np.where(topCourses == course)
    if not encode[0]:
      encode = -1
    else:
      encode = encode[0][0]
    return encode

In [ ]:
#generate coordinates for one course
def generate_coordinates(course, prereq):
  x = [encode_course(course)] * len(prereq)
  y = [encode_course(i) for i in prereq]
  return x, y


In [ ]:
#generate scatter plot

results = pd.read_pickle(DATA_PATH+"/topResultsWith230.pkl")
prereq_db = pickle.load(open(DATA_PATH+"/prereq_db.pkl", "rb"))
f = plt.figure()

for row_index, row in results.iterrows():
  course = row['course_name']
  #draw official prereqs
  dp_code = re.findall("[A-Z]+", course)[0]
  official_prereq = [dp_code + str(i) for i in prereq_db[course]]
  x_0, y_0 = generate_coordinates(course, official_prereq)
  plt.scatter(x_0, y_0, c='r')

  #draw prereq in coefs
  prereq_in_coefs = row['official_prereq_in_coefs']
  x_1, y_1 = generate_coordinates(course, prereq_in_coefs)
  plt.scatter(x_1, y_1, c='y')

  #draw coefs
  coefs = row['coefs']
  coefs_courses = [i[0] for i in coefs]
  coefs_weights = [i[1] for i in coefs]
  x_2, y_2 = generate_coordinates(course, coefs_courses)
  plt.scatter(x_2, y_2, c='g', s=coefs_weights)

ax = plt.subplot(1,1,1)
ax.set_ylim(bottom=0.)
ax.set_xlim(left=0.)
plt.xlabel("top 40 Courses")
plt.ylabel("prereqs")
ax.set_xticks([24,41])
ax.set_xticklabels(['CS229','CS230'])
plt.yticks([])
legend_elements = [Line2D([0], [0], marker='o', color='w', label='official_prereq',
                          markerfacecolor='r', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='predict_coefs',
                          markerfacecolor='g', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='official_prereq_in_coefs',
                          markerfacecolor='y', markersize=15)
                   ]
ax.legend(handles=legend_elements)
plt.show()

f.savefig(DATA_PATH+"/result.pdf", bbox_inches='tight')




